<a href="https://colab.research.google.com/github/GonzaloPerezGomez/Pytorch/blob/main/03_Computer_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>